In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "..", "utilities")))
from format import process_federated_dataset,process_specific_datasets_and_files

# federated_endpoint_sets_to_test = ["Uniprot"]
# # Process the federated dataset and get the output directory
# output_dir = process_federated_dataset(
#     endpoint_sets_to_test=federated_endpoint_sets_to_test
# )

# print(f"Processing complete. Results stored in: {output_dir}")

In [9]:
result_dir = process_specific_datasets_and_files(
    endpoint_files_map={"Uniprot": ["48_glycosylation_sites_and_glycans.ttl", "50_tissues_where_genes_sphingosines_are_expressed.ttl"]}
)

print(f"Processing complete. Results stored in: {result_dir}")

Processing endpoint set: Uniprot

Processed: 50_tissues_where_genes_sphingosines_are_expressed.ttl

Processed: 48_glycosylation_sites_and_glycans.ttl
Processed 2 files from Uniprot
Saved metadata for 2 files from 1 endpoint sets to:
  - /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-07_13-57-51/testset_meta_data.json
Processing complete. Results stored in: /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-07_13-57-51


In [6]:
from datasets import Dataset
from format import load_data_from_file

test_set = load_data_from_file(os.path.join(output_dir, "testset_meta_data.json"))

test_dataset = Dataset.from_dict(test_set)

print(test_dataset[0:1])

{'resource': ['https://sparql.uniprot.org/.well-known/sparql-examples/118_biosodafrontend_rat_brain_human_cancer'], 'natural_language_question': ['What are the Homo sapiens genes associated with cancer and their orthologs expressed in the Rattus norvegicus brain?'], 'query': ['PREFIX up:<http://purl.uniprot.org/core/>\nPREFIX taxon:<http://purl.uniprot.org/taxonomy/>\nPREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>\nPREFIX orth:<http://purl.org/net/orth#>\nPREFIX dcterms:<http://purl.org/dc/terms/>\nPREFIX obo:<http://purl.obolibrary.org/obo/>\nPREFIX lscr:<http://purl.org/lscr#>\nPREFIX genex:<http://purl.org/genex#>\nPREFIX sio: <http://semanticscience.org/resource/>\nSELECT ?gene ?orthologous_protein2 WHERE {\n  {\n    SELECT ?protein1 WHERE {\n      ?protein1 a up:Protein;\n        up:organism/up:scientificName \'Homo sapiens\' ;\n        up:annotation ?annotation .\n      ?annotation rdfs:comment ?annotation_text.\n      ?annotation a up:Disease_Annotation .\n      FILTER CONT

In [ ]:
from langsmith import Client
from datetime import datetime

client = Client()
runs = client.list_runs(project_name="sparql-rag-agent", is_root=True)
first_run = next(runs)


child_runs = list(client.list_runs(project_name="sparql-rag-agent", parent_run_id=first_run.id))

#print(child_runs)

# Filter to find the sparql_query_construction run
sparql_construction_run = None
for run in child_runs:
    if run.name == "sparql_query_construction":
        sparql_construction_run = run
        break

if sparql_construction_run:
    result = {
        # Your existing fields...
        "final_state_response": "",
        "run_id_langsmith": str(first_run.id),
        # Add the specific metrics for the sparql_query_construction run
        "sparql_construction_prompt_tokens": sparql_construction_run.prompt_tokens or 0,
        "sparql_construction_completion_tokens": sparql_construction_run.completion_tokens or 0,
        "sparql_construction_total_tokens": sparql_construction_run.total_tokens or 0,
        "sparql_construction_prompt_cost": sparql_construction_run.prompt_cost or 0,
        "sparql_construction_completion_cost": sparql_construction_run.completion_cost or 0,
        "sparql_construction_total_cost": sparql_construction_run.total_cost or 0,
        # Keep the original metrics too if needed
        "prompt_tokens": first_run.prompt_tokens or 0,
        "completion_tokens": first_run.completion_tokens or 0,
        "total_tokens": first_run.total_tokens or 0,
        "prompt_cost": first_run.prompt_cost or 0,
        "completion_cost": first_run.completion_cost or 0,
        "total_cost": first_run.total_cost or 0,
    }
else:
    print("Warning: Could not find the sparql_query_construction run")
    # Fall back to using the parent run metrics
    result = {
        # Original metrics...
    }
print(result)

# # === BASIC INFORMATION ===
# print("=== BASIC INFORMATION ===")
# print(f"ID: {first_run.id}")
# print(f"Name: {first_run.name}")
# print(f"Run Type: {first_run.run_type}")
# print(f"Status: {first_run.status}")
# print(f"App Path: {first_run.app_path}")

# # === TIMING INFORMATION ===
# print("\n=== TIMING INFORMATION ===")
# print(f"Start Time: {first_run.start_time}")
# print(f"End Time: {first_run.end_time}")
# print(f"First Token Time: {first_run.first_token_time}")
# print(f"Execution Time: {first_run.end_time - first_run.start_time}")

# # === TOKEN AND COST METRICS ===
# print("\n=== TOKEN AND COST METRICS ===")
# print(f"Prompt Tokens: {first_run.prompt_tokens}")
# print(f"Completion Tokens: {first_run.completion_tokens}")
# print(f"Total Tokens: {first_run.total_tokens}")
# print(f"Prompt Cost: ${first_run.prompt_cost}")
# print(f"Completion Cost: ${first_run.completion_cost}")
# print(f"Total Cost: ${first_run.total_cost}")

# # === INPUT/OUTPUT DATA ===
# print("\n=== INPUT/OUTPUT DATA ===")
# print(f"Inputs: {first_run.inputs}")
# print(f"Outputs: {first_run.outputs}")
# print(f"Error: {first_run.error}")

# # === RELATIONSHIP PROPERTIES ===
# print("\n=== RELATIONSHIP PROPERTIES ===")
# print(f"Parent Run ID: {first_run.parent_run_id}")
# print(f"Parent Run IDs: {first_run.parent_run_ids}")
# print(f"Child Run IDs: {first_run.child_run_ids}")
# print(f"Child Runs: {first_run.child_runs}")
# print(f"Trace ID: {first_run.trace_id}")
# print(f"Dotted Order: {first_run.dotted_order}")

# # === METADATA ===
# print("\n=== METADATA ===")
# print(f"Tags: {first_run.tags}")
# print(f"Extra: {first_run.extra}")
# print(f"Serialized: {first_run.serialized}")
# print(f"Events: {first_run.events}")
# print(f"Feedback Stats: {first_run.feedback_stats}")
# print(f"In Dataset: {first_run.in_dataset}")
# print(f"Reference Example ID: {first_run.reference_example_id}")
# print(f"Session ID: {first_run.session_id}")
# print(f"Manifest ID: {first_run.manifest_id}")

AttributeError: 'Run' object has no attribute 'model'

In [ ]:
import asyncio
import sys
sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.evaluation import AgentEvaluator


evaluator = AgentEvaluator(
    endpoint_sets={"Uniprot": ["48_glycosylation_sites_and_glycans.ttl"]},
    test=True,
    experiment_dir="experiments_official"
)

await evaluator.run_all_tests()

Processing endpoint set: Uniprot

Processed: 48_glycosylation_sites_and_glycans.ttl
Processed 1 files from Uniprot
Saved metadata for 1 files from 1 endpoint sets to:
  - /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-08_19-54-15/testset_meta_data.json
Sending question 1/1 to agent...
{'messages': [HumanMessage(content='Retrieve glycosylation sites and glycans on human enzymes (federated with Glyconnect)', additional_kwargs={}, response_metadata={}, id='20d0e730-cfba-4f9d-8838-d1a0e23e07c3')], 'structured_question': StructuredQuestion(intent='access_resources', extracted_classes=['glycosylation sites', 'glycans', 'enzymes'], extracted_entities=['human', 'Glyconnect'], question_steps=['Retrieve glycosylation sites on human enzymes', 'Retrieve glycans on human enzymes']), 'retrieved_docs': [], 'extracted_entities': [{'matchs': [Document(metadata={'uri': 'http://purl.obolibrary.org/obo/CHEBI_30879', 'type': 'literal', 'description'

'/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-08_19-54-15/evaluation_dataset.json'